## Using *checkpoint_schedules* package

This first example aim to present a quick start of the *checkpoint_schedules* usage. Thus, the example is a basic illustration showing how this package woks and how to read the time execution using this package.

To execute a adjoint-based gradient problem with *checkpoint_schedules*, we initially implement `CheckpointingManager` class built to handle both forward and adjoint executions. This management is achieved by iterating over a sequence of schedules through the execution of `CheckpointingManager.execute(cp_schedule)`. The `cp_schedule` must be a *checkpoint_schedules* objective with a generator method that allows the execution of the forward and adjoint solvers driven by the schedules of actions: *Forward, EndForward, Reverse, Copy, Move, EndReverse.*

Whitin `CheckpointingManager.execute` method, we define the actions using single-dispatch functions, with the `action` function being the generic function decorated with the `singledispatch` function. Specific functions for the *checkpoint_schedules* actions are provided through the register method of the `action` base function.

In [21]:
from checkpoint_schedules import Forward, EndForward, Reverse, Copy, Move, EndReverse
import functools

class CheckpointingManager():
    """Manage the forward and adjoint solvers.

    Attributes
    ----------
    save_ram : int
        Number of checkpoint that will be stored in `'RAM'`.
    save_disk : int
        Number of checkpoint that will be stored on `'disk'`.
    list_actions : list
        Store the list of actions.
    max_n : int
        Total steps used to execute the solvers.
    """
    def __init__(self, max_n, chk_ram=0, chk_disk=0):
        self.max_n = max_n
        self.save_ram = chk_ram
        self.save_disk = chk_disk
        self.list_actions = []
        
    def execute(self, cp_schedule):
        """Execute forward and adjoint with a checkpointing strategy.

        Parameters
        ----------
        cp_schedule : CheckpointSchedule
            Checkpointing schedule object.
        """
        @functools.singledispatch
        def action(cp_action):
            raise TypeError("Unexpected action")

        @action.register(Forward)
        def action_forward(cp_action):
            nonlocal model_n
            n1 = min(cp_action.n1, self.max_n)
            if cp_action.write_ics:
                print((" +").rjust(cp_action.n0*4))
            if cp_action.write_adj_deps:
                print(("+").rjust(n1*4))
   
            print(("|" + "--->"*(n1-cp_action.n0)).rjust(n1*4) +
                   "   "*(self.max_n - n1 + 4) + 
                   self.list_actions[len(self.list_actions) - 1])

            model_n = n1
            if n1 == self.max_n:
                cp_schedule.finalize(n1)

        @action.register(Reverse)
        def action_reverse(cp_action):
            nonlocal model_r
            print(("<---"*(cp_action.n1-cp_action.n0) + "|").rjust(cp_action.n1*4) 
                  + "   "*(self.max_n - cp_action.n1 + 4) + 
                    self.list_actions[len(self.list_actions) - 1])

            model_r += cp_action.n1 - cp_action.n0
            
        @action.register(Copy)
        def action_copy(cp_action):
            print(("c").rjust(cp_action.n*4) 
                  + "   "*(self.max_n + 4) + 
                    self.list_actions[len(self.list_actions) - 1])

        @action.register(Move)
        def action_move(cp_action):
            print(("-").rjust(cp_action.n*4) 
                  + "   "*(self.max_n + 3) + 
                    self.list_actions[len(self.list_actions) - 1])

        @action.register(EndForward)
        def action_end_forward(cp_action):
            assert model_n == self.max_n
            # The correct number of adjoint steps has been taken
            if cp_schedule._max_n is None:
                cp_schedule._max_n = self.max_n
            
        @action.register(EndReverse)
        def action_end_reverse(cp_action):
            nonlocal model_r
            assert model_r == self.max_n
            print("End Reverse")

        model_n = 0
        model_r = 0

        for count, cp_action in enumerate(cp_schedule):
            self.list_actions.append(str(cp_action))
            action(cp_action)
            if isinstance(cp_action, EndReverse):  
                break

Notice that `CheckpointingManager` constructor is optional to define the number of checkpoint steps to store in ram (`chk_ram`) and in disk (`chk_disk`). The reason is that *checkpointing_schedules* works for the case where no adjoint calculation is performed. Therefore, there is no need of storing any forward checkpointing data. This type of schedule is available to make easier the integration of this package with an adjoint-based gradient solver which is not restrict on applying the a checkpointing strategy. To exemplify this case of no adjoint computation, let us impose the number of total steps `max_n` and the object to drive the the forward solver (`solver_manager`). 

In [22]:
max_n = 4 # Total number of time steps.
solver_manager = CheckpointingManager(max_n) # manager object

The checkpoint schedule for the case where there is no adjoint calculation is give by `checkpoint_schedules.NoneCheckpointSchedule`. Below we have illustrated the employment of this schedule for the time steps (`max_n`) defined above.

In [23]:
from checkpoint_schedules import NoneCheckpointSchedule
cp_schedule = NoneCheckpointSchedule() # checkpoint schedule object
solver_manager.execute(cp_schedule)

|--->--->--->--->            Forward(0, 9223372036854775807, False, False, <StorageType.NONE: None>)


The ouputs of the `solver_manager.execute(cp_schedule)` illustrate the execution in time on the right side and print *checkpoint_schedules* action on the left side.

To clarify how the `Forward` action works, consider initially the general form: 
    
    * *Forward(n0, n1, write_ics, write_adj_deps, 'storage')*:

        - Advance the forward solver from the step `n0`` to the start of the step `n1`.
        - Store the forward data required to initialise the forward solver from the step `n0` if `write_ics` is `'True'`.
        - Write the forward data required for the adjoint computation from the step `n1` to the step `n0` if `write_adj_deps` is `'True'`.
        - Storage type to save the forward data.

Therefore, for this particular case, we have:

    * Forward(0, 9223372036854775807, False, False, <StorageType.NONE: None>), which reads:
    


In [25]:
from checkpoint_schedules import SingleMemoryStorageSchedule

cp_schedule = SingleMemoryStorageSchedule()
solver_manager.execute(cp_schedule)

 +
               +
|--->--->--->--->            Forward(0, 9223372036854775807, True, True, <StorageType.WORKING_MEMORY: 4>)
<---<---<---<---|            Reverse(4, 0, True)
End Reverse


In [28]:
from checkpoint_schedules import Revolve
chk_ram = 2
solver_manager = CheckpointingManager(max_n, chk_ram=chk_ram) # manager object
cp_schedule = Revolve(max_n, chk_ram)
solver_manager.execute(cp_schedule)

 +
|--->--->                  Forward(0, 2, True, False, <StorageType.RAM: 0>)
       +
       |--->               Forward(2, 3, True, False, <StorageType.RAM: 0>)
               +
           |--->            Forward(3, 4, False, True, <StorageType.ADJ_DEPS: 3>)
           <---|            Reverse(4, 3, True)
       -                     Move(2, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
           +
       |--->               Forward(2, 3, False, True, <StorageType.ADJ_DEPS: 3>)
       <---|               Reverse(3, 2, True)
c                        Copy(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
|--->                     Forward(0, 1, False, False, <StorageType.FWD_RESTART: 2>)
       +
   |--->                  Forward(1, 2, False, True, <StorageType.ADJ_DEPS: 3>)
   <---|                  Reverse(2, 1, True)
-                     Move(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
   +
|--->                     Forward(0, 1, False, True, <StorageType.AD

In [31]:
from checkpoint_schedules import HRevolve
chk_disk = 1
chk_ram = 1
solver_manager = CheckpointingManager(max_n, chk_ram=chk_ram, chk_disk=chk_disk) # manager object
cp_schedule = HRevolve(max_n, chk_ram, snap_on_disk=chk_disk)
solver_manager.execute(cp_schedule)

 +
|--->--->--->               Forward(0, 3, True, False, <StorageType.RAM: 0>)
               +
           |--->            Forward(3, 4, False, True, <StorageType.ADJ_DEPS: 3>)
           <---|            Reverse(4, 3, True)
c                        Copy(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
|--->--->                  Forward(0, 2, False, False, <StorageType.FWD_RESTART: 2>)
           +
       |--->               Forward(2, 3, False, True, <StorageType.ADJ_DEPS: 3>)
       <---|               Reverse(3, 2, True)
c                        Copy(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
|--->                     Forward(0, 1, False, False, <StorageType.FWD_RESTART: 2>)
       +
   |--->                  Forward(1, 2, False, True, <StorageType.ADJ_DEPS: 3>)
   <---|                  Reverse(2, 1, True)
-                     Move(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
   +
|--->                     Forward(0, 1, False, True, <StorageType.ADJ_D

In [32]:
from checkpoint_schedules import DiskRevolve
solver_manager = CheckpointingManager(max_n, chk_ram=chk_ram) # manager object``
cp_schedule = DiskRevolve(max_n, chk_ram)
solver_manager.execute(cp_schedule)

 +
|--->--->--->               Forward(0, 3, True, False, <StorageType.RAM: 0>)
               +
           |--->            Forward(3, 4, False, True, <StorageType.ADJ_DEPS: 3>)
           <---|            Reverse(4, 3, True)
c                        Copy(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
|--->--->                  Forward(0, 2, False, False, <StorageType.FWD_RESTART: 2>)
           +
       |--->               Forward(2, 3, False, True, <StorageType.ADJ_DEPS: 3>)
       <---|               Reverse(3, 2, True)
c                        Copy(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
|--->                     Forward(0, 1, False, False, <StorageType.FWD_RESTART: 2>)
       +
   |--->                  Forward(1, 2, False, True, <StorageType.ADJ_DEPS: 3>)
   <---|                  Reverse(2, 1, True)
-                     Move(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
   +
|--->                     Forward(0, 1, False, True, <StorageType.ADJ_D

In [33]:
from checkpoint_schedules import PeriodicDiskRevolve
cp_schedule = PeriodicDiskRevolve(max_n, chk_ram)
solver_manager.execute(cp_schedule)

We use periods of size  3
 +
|--->--->--->               Forward(0, 3, True, False, <StorageType.RAM: 0>)
               +
           |--->            Forward(3, 4, False, True, <StorageType.ADJ_DEPS: 3>)
           <---|            Reverse(4, 3, True)
c                        Copy(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
|--->--->                  Forward(0, 2, False, False, <StorageType.FWD_RESTART: 2>)
           +
       |--->               Forward(2, 3, False, True, <StorageType.ADJ_DEPS: 3>)
       <---|               Reverse(3, 2, True)
c                        Copy(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
|--->                     Forward(0, 1, False, False, <StorageType.FWD_RESTART: 2>)
       +
   |--->                  Forward(1, 2, False, True, <StorageType.ADJ_DEPS: 3>)
   <---|                  Reverse(2, 1, True)
-                     Move(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
   +
|--->                     Forward(0, 1, False

In [34]:
from checkpoint_schedules import MixedCheckpointSchedule

cp_schedule = MixedCheckpointSchedule(max_n, chk_disk)
solver_manager.execute(cp_schedule)

 +
|--->--->--->               Forward(0, 3, True, False, <StorageType.DISK: 1>)
               +
           |--->            Forward(3, 4, False, True, <StorageType.ADJ_DEPS: 3>)
           <---|            Reverse(4, 3, True)
c                        Copy(0, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
|--->--->                  Forward(0, 2, False, False, <StorageType.FWD_RESTART: 2>)
           +
       |--->               Forward(2, 3, False, True, <StorageType.ADJ_DEPS: 3>)
       <---|               Reverse(3, 2, True)
-                     Move(0, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
   +
|--->                     Forward(0, 1, False, True, <StorageType.DISK: 1>)
       +
   |--->                  Forward(1, 2, False, True, <StorageType.ADJ_DEPS: 3>)
   <---|                  Reverse(2, 1, True)
-                     Move(0, <StorageType.DISK: 1>, <StorageType.ADJ_DEPS: 3>)
<---|                     Reverse(1, 0, True)
End Reverse


/home/ma/d/ddolci/work/checkpoint_schedules/checkpoint_schedules/mixed.py:59: RuntimeWarning: Numba not available -- using memoization
  warnings.warn("Numba not available -- using memoization",


In [35]:
from checkpoint_schedules import MultistageCheckpointSchedule

cp_schedule = MultistageCheckpointSchedule(max_n, 0, chk_disk)
solver_manager.execute(cp_schedule)

 +
|--->--->--->               Forward(0, 3, True, False, <StorageType.DISK: 1>)
               +
           |--->            Forward(3, 4, False, True, <StorageType.ADJ_DEPS: 3>)
           <---|            Reverse(4, 3, True)
c                        Copy(0, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
|--->--->                  Forward(0, 2, False, False, <StorageType.FWD_RESTART: 2>)
           +
       |--->               Forward(2, 3, False, True, <StorageType.ADJ_DEPS: 3>)
       <---|               Reverse(3, 2, True)
c                        Copy(0, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
|--->                     Forward(0, 1, False, False, <StorageType.FWD_RESTART: 2>)
       +
   |--->                  Forward(1, 2, False, True, <StorageType.ADJ_DEPS: 3>)
   <---|                  Reverse(2, 1, True)
-                     Move(0, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
   +
|--->                     Forward(0, 1, False, True, <StorageType.A

In [36]:
from checkpoint_schedules import TwoLevelCheckpointSchedule
revolver = TwoLevelCheckpointSchedule(2, chk_disk)
solver_manager.execute(revolver)


 +
|--->--->                  Forward(0, 2, True, False, <StorageType.DISK: 1>)
       +
       |--->--->            Forward(2, 4, True, False, <StorageType.DISK: 1>)
       c                        Copy(2, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
       |--->               Forward(2, 3, False, False, <StorageType.FWD_RESTART: 2>)
               +
           |--->            Forward(3, 4, False, True, <StorageType.ADJ_DEPS: 3>)
           <---|            Reverse(4, 3, True)
       c                        Copy(2, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
           +
       |--->               Forward(2, 3, False, True, <StorageType.ADJ_DEPS: 3>)
       <---|               Reverse(3, 2, True)
c                        Copy(0, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
|--->                     Forward(0, 1, False, False, <StorageType.FWD_RESTART: 2>)
       +
   |--->                  Forward(1, 2, False, True, <StorageType.ADJ_DEPS: 3>)
   <---|       